In [143]:
import config
import pathlib
import json

In [133]:
labels_json_path = list(pathlib.Path(config.data_raw_dev).rglob("*.json"))[0]

In [134]:
with open(labels_json_path, "r") as f:
    data = json.load(f)

## Not all of the questions are for Task 1

for example

  {'Question': 'Where exactly in the image is the instrument located?',
   'AnswerType': 'segmentation',
   'Answer': 'clb0lbwzadoyc086u0brshvx5_mask.png'}

   is not 

In [135]:
questions_to_be_answered = ["What type of procedure is the image taken from?",
"How many instrumnets are in the image?",
"Have all polyps been removed?",
"Where in the image is the abnormality?",
"Is this finding easy to detect?",
"Where in the image is the instrument?",
"Is there a green/black box artifact?",
"Are there any abnormalities in the image?",
"Is there text?",
"Are there any anatomical landmarks in the image?",
"What color is the abnormality?",
"Are there any instruments in the image?",
"What color is the anatomical landmark?",
"Where in the image is the anatomical landmark?",
"How many findings are present?",
"What is the size of the polyp?",
"How many polyps are in the image?",
"What type of polyp is present?"]

In [136]:
question_answers = {}
for image in data:
    for label in image["Labels"]:
        if label["Question"] in questions_to_be_answered:
            if label["Question"] not in question_answers:
                question_answers[label["Question"]] = []
            
            # Add answers to the question , then make sure it wasn't added multiple times
            question_answers[label["Question"]] +=  [label["Question"] + "_" + answer for answer in label["Answer"]]  
            question_answers[label["Question"]] = list(set(question_answers[label["Question"]]))

Answers seem good, besides that there are two colors in "How many findings are present?"

In [137]:
# for image in data:
#     for label in image["Labels"]:
#         if label["Question"] in questions_to_be_answered:
#             if label["Question"] == "How many findings are present?":
#                 print(image["ImageID"], label["Answer"])

There are some bad labels in there...
How many is not a question you can answer with pink, yellow 
But we'll just leave it in for now, the model should never give it as an answer. probably

In [138]:
labels = []
answers = list(question_answers.values())
for q in answers:
    labels.extend(q)
labels

['Are there any abnormalities in the image?_Ulcerative colitis',
 'Are there any abnormalities in the image?_Oesophagitis',
 'Are there any abnormalities in the image?_Polyp',
 'Are there any abnormalities in the image?_Barretts',
 'Are there any abnormalities in the image?_No',
 'Are there any anatomical landmarks in the image?_Pylorus',
 'Are there any anatomical landmarks in the image?_Ileum',
 'Are there any anatomical landmarks in the image?_No',
 'Are there any anatomical landmarks in the image?_Cecum',
 'Are there any anatomical landmarks in the image?_Z-line',
 'Are there any anatomical landmarks in the image?_Not relevant',
 'Are there any instruments in the image?_Injection needle',
 'Are there any instruments in the image?_No',
 'Are there any instruments in the image?_Tube',
 'Are there any instruments in the image?_Polyp snare',
 'Are there any instruments in the image?_Biopsy forceps',
 'Are there any instruments in the image?_Metal clip',
 'Are there any instruments in t

# Encoding

For example if an image has the question - answer
 'Are there any instruments in the image?_Polyp snare',
 'Are there any instruments in the image?_Biopsy forceps'

There should be a vector with those two values = 1

In [139]:
sample_labels = [
    ['What color is the abnormality?_Yellow', 'What is the size of the polyp?_< 5mm'],
    ['What color is the anatomical landmark?_Red', 'What is the size of the polyp?_11-20mm'],
    ['What color is the abnormality?_grey', 'What color is the anatomical landmark?_White']
]

In [140]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([labels])
mlb.classes_[0:10]

array(['Are there any abnormalities in the image?_Barretts',
       'Are there any abnormalities in the image?_No',
       'Are there any abnormalities in the image?_Oesophagitis',
       'Are there any abnormalities in the image?_Polyp',
       'Are there any abnormalities in the image?_Ulcerative colitis',
       'Are there any anatomical landmarks in the image?_Cecum',
       'Are there any anatomical landmarks in the image?_Ileum',
       'Are there any anatomical landmarks in the image?_No',
       'Are there any anatomical landmarks in the image?_Not relevant',
       'Are there any anatomical landmarks in the image?_Pylorus'],
      dtype=object)

In [141]:
encoded_samples  = mlb.transform(sample_labels)
encoded_samples 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [142]:
answer = mlb.inverse_transform(encoded_samples)
answer

[('What color is the abnormality?_Yellow',
  'What is the size of the polyp?_< 5mm'),
 ('What color is the anatomical landmark?_Red',
  'What is the size of the polyp?_11-20mm'),
 ('What color is the abnormality?_grey',
  'What color is the anatomical landmark?_White')]